In [1]:
!pip install deap numpy pandas


In [2]:
!pip install deap numpy pandas

import numpy as np
import pandas as pd
import random
from deap import base, creator, tools, algorithms
import json


In [3]:
import random
import json
import numpy as np
from itertools import product

# Definir los indicadores y sus posibles estados
indicators = [
    "Activar Absolute Strength (Histograma)",
    "Activar SSL Channel",
    "Activar RSI",
    "Activar Squeeze Momentum",
    "Activar MACD Signal",
    "Activar MACD Histograma",
    "Activar Condiciones MTF",
    "Activar Condiciones EMAs",
    "Activar Distancia entre EMAs",
    "Usar Firestorm"
    ]

# Indicadores que pueden ser "Excluyentes"
excluyentes_permitidos = [
    "Activar Absolute Strength (Histograma)",
    "Activar SSL Channel",
    "Activar Condiciones MTF",
    "Activar Squeeze Momentum",
    "Activar Distancia entre EMAs",
    "Usar Firestorm"
]

# Definir los estados válidos para los indicadores
indicator_states_permitidos = ["Excluyente", "Opcional", "Desactivado"]
indicator_states_generales = ["Opcional", "Desactivado"]

# Generar combinaciones de estados para los indicadores
def generate_combinations():
    combinations = []

    for states in product(indicator_states_generales, repeat=len(indicators) - len(excluyentes_permitidos)):
        for excl_states in product(indicator_states_permitidos, repeat=len(excluyentes_permitidos)):
            combination = dict(zip(indicators, excl_states + states))

            # Contar el número de indicadores opcionales y excluyentes
            num_excluyente = sum(1 for key, value in combination.items() if value == "Excluyente")
            num_opcional = sum(1 for value in combination.values() if value == "Opcional")

            # Validar que solo los indicadores permitidos puedan ser excluyentes
            valid_exclusives = all(
                key in excluyentes_permitidos or value != "Excluyente"
                for key, value in combination.items()
            )

            # Validar el número de excluyentes y opcionales según las reglas
            if valid_exclusives and 1 <= num_excluyente <= 3:
                if num_excluyente == 3 and 3 <= num_opcional <= 8:
                    combinations.append(combination)
                elif num_excluyente == 2 and 4 <= num_opcional <= 9:
                    combinations.append(combination)
                elif num_excluyente == 1 and 4 <= num_opcional <= 10:
                    combinations.append(combination)
    
    return combinations

# Generar combinaciones de indicadores antes de aplicar la gestión de riesgos
valid_combinations = generate_combinations()
print(f"Total de combinaciones de indicadores antes de la gestión de riesgos: {len(valid_combinations)}")

def apply_requires(combinations):
    final_combinations = []
    
    for combination in combinations:
        num_opcional = sum(1 for value in combination.values() if value == "Opcional")
        num_excluyente = sum(1 for value in combination.values() if value == "Excluyente")
        
        if num_excluyente == 1:
            if num_opcional in [4, 5]:
                min_requerido = int(round(0.65 * num_opcional))
                max_requerido = int(round(0.78 * num_opcional))
            elif num_opcional >= 6:
                min_requerido = int(round(0.6 * num_opcional))
                max_requerido = int(round(0.78 * num_opcional))
        elif num_excluyente == 2:
            if num_opcional in [4, 5]:
                min_requerido = int(round(0.51 * num_opcional))
                max_requerido = int(round(0.78 * num_opcional))
            elif num_opcional >= 6:
                min_requerido = int(round(0.57 * num_opcional))
                max_requerido = int(round(0.78 * num_opcional))
        elif num_excluyente == 3:
            min_requerido = int(round(0.43 * num_opcional))
            max_requerido = int(round(0.73 * num_opcional))

        # Generar valores de gestión de riesgo dentro de los límites válidos
        for num_opcional_requeridos in range(min_requerido, max_requerido + 1):
            requires = {
                "Número de Indicadores Opcionales requeridos": int(num_opcional_requeridos)  # Conversión a int
            }

            final_combinations.append({
                "indicators": combination,
                "requires": requires
            })
    
    return final_combinations

# Aplicar la función a las combinaciones válidas
combinations_with_requires = apply_requires(valid_combinations)

print(f"Total de combinaciones después de aplicar los opcionales requeridos: {len(combinations_with_requires)}")


Total de combinaciones de indicadores antes de la gestión de riesgos: 1734
Total de combinaciones después de aplicar los opcionales requeridos: 3199


In [4]:

def apply_risk_management(combinations):
    profit_take_values = np.arange(25, 35, 5)
    take_profit_multiplier_values = np.round(np.arange(0.35, 0.45, 0.5), 2)
    
    final_combinations = []
    num_combinations = len(combinations)
    segments = {1: [], 2: [], 3: []}

    # Segmentar las combinaciones por el número de excluyentes
    for combination in combinations:
        num_excluyente = sum(1 for value in combination['indicators'].values() if value == "Excluyente")
        segments[num_excluyente].append(combination)

    # Calcular cuántas combinaciones de cada segmento queremos mantener
    target_per_segment = num_combinations // 3

    for excluyentes, segment_combinations in segments.items():
        sampled_combinations = random.sample(segment_combinations, min(target_per_segment, len(segment_combinations)))
        for combination in sampled_combinations:
            for profit_take in profit_take_values:
                for multiplier in take_profit_multiplier_values:
                    risk_management = {
                        "Porcentaje de toma de ganancias": int(profit_take),  # Conversión a int
                        "Multiplier for Take Profit": float(multiplier),  # Conversión a float
                    }
                    new_combination = combination.copy()
                    new_combination['riskManagement'] = risk_management
                    final_combinations.append(new_combination)
    
    return final_combinations[:num_combinations]  # Asegurar que el total no exceda las combinaciones originales

# Aplicar la función a las combinaciones válidas
combinations_with_risk_management = apply_risk_management(combinations_with_requires)

print(f"Total de combinaciones después de aplicar el risk management: {len(combinations_with_risk_management)}")

# Exportar las combinaciones finales a un archivo JSON
output_filename = 'final_combinations.json'
with open(output_filename, 'w') as f:
    json.dump(combinations_with_risk_management, f, indent=2)

print(f"Combinaciones exportadas a {output_filename}")


Total de combinaciones después de aplicar el risk management: 3199
Combinaciones exportadas a final_combinations.json


In [5]:
import json

# Crear el diccionario con las combinaciones
final_combinations = {}  # Usar un diccionario en lugar de una lista

for idx, combination in enumerate(combinations_with_risk_management, start=1):
    # Crear el nombre de la combinación en el formato solicitado
    combination_name = f"combination_{idx + 1}"  # Ajustar el inicio del índice según sea necesario

    # Añadir la combinación completa al diccionario final
    final_combinations[combination_name] = {
        "indicators": combination['indicators'],
        "riskManagement": combination['riskManagement'],  # Asegurarse de incluir el riskManagement
        "requires": combination['requires']
    }

# Guardar el archivo JSON con el formato deseado
with open('final_combinations.json', 'w') as f:
    json.dump(final_combinations, f, indent=2)

print(f"Total de combinaciones guardadas: {len(final_combinations)}")


Total de combinaciones guardadas: 3199


In [6]:
import json
from collections import Counter

# Cargar el archivo JSON exportado
with open('/kaggle/working/final_combinations.json', 'r') as f:
    combinations_with_risk_management = json.load(f)

# Contar las ocurrencias de cada valor en "Porcentaje de toma de ganancias" y "Multiplier for Take Profit"
profit_take_counter = Counter()
multiplier_counter = Counter()

# Iterar sobre las combinaciones correctamente
for key, combination in combinations_with_risk_management.items():
    profit_take = combination['riskManagement']['Porcentaje de toma de ganancias']
    multiplier = combination['riskManagement']['Multiplier for Take Profit']
    profit_take_counter[profit_take] += 1
    multiplier_counter[multiplier] += 1

# Mostrar los resultados en la consola
print("Desglose de 'Porcentaje de toma de ganancias':")
for value, count in profit_take_counter.items():
    print(f"  {value}: {count} combinaciones")

print("\nDesglose de 'Multiplier for Take Profit':")
for value, count in multiplier_counter.items():
    print(f"  {value}: {count} combinaciones")


Desglose de 'Porcentaje de toma de ganancias':
  25: 1600 combinaciones
  30: 1599 combinaciones

Desglose de 'Multiplier for Take Profit':
  0.35: 3199 combinaciones
